In [1]:
import rs_classes.async_request_client as rs

token = ""
base_url = "https://us.app.rossum.ai/api"
annotation_list = ["7700996"]
target_schema_id = "vendor_discount_size"

client = rs.AsyncRequestClient(token, base_url)

In [ ]:
import rs_classes.hooks as hs
import rs_functions.fetch_annotations_meta as fetch_annotations_meta
import rs_functions.fetch_annotation_content as fetch_annotation_content
import rs_functions.mongo_viewer_functions as mvf

annotations_collection = await fetch_annotations_meta.get_annotation_meta(client, annotation_list)
await fetch_annotation_content.get_annotation_content(client, annotations_collection)
hooks = await mvf.collect_hooks_per_annotation(client, annotations_collection)


In [ ]:
# WORK IN PROGRESS
# let's try to get all loops



for annotation_id, annotation in annotations_collection.items():

    # in this place we should have multiple blocks with UI analysis
    mdh_hooks = []
    for hook in annotation.related_hooks:
        
        hook_obj =hooks.get_hook(hook.split("/")[-1])
        if hook_obj.hook_template == "https://us.app.rossum.ai/api/v1/hook_templates/39":
            mdh_hooks.append(hook)            
            for configuration in hook_obj.settings.get("configurations", None):                
                if target_schema_id == configuration["mapping"]["target_schema_id"]:
                    print(f"\033[33m Analysing annotation: {annotation_id}.. \033[0m")
                    print( f"\033[36m Hook {hook_obj.id} - {hook_obj.name} has configuration with target_schema_id = {target_schema_id}\033[0m")                                        
                    

                    # this is where we need to extract the query and start working with every one.   But before we need to create API calls to use our MongoDB.              
                    # deconstruct queries? 
                    for query in configuration["source"]["queries"]:

                        result = None
                        ### FIND ALL PLACEHOLDERS AND REPLACE THEM WITH REAL VALUES. 


                        if query.get("find"):
                            print("find")
                            ## Go with "find"

                            find = await client.data_storage_find(collectionName=configuration["source"]["dataset"], query=query["find"])
                            print(find)


                        elif query.get("aggregate"):
                            print("aggregate")
                            
                            aggregate = await client.data_storage_aggregate(collectionName=configuration["source"]["dataset"], pipeline=query)
                            # here the aggregation must be further split by pipeline stages. However for now let's check them as they are. 
                            

                        else:
                            print("No Find or Aggregate has been found")

                        
                        # check result and if it is not empty break the loop. 
                        

    # print(mdh_hooks)
        
        



    
